In [1]:
from Acts.extractor import getCopora

In [2]:
corpora = getCopora()
print(len(corpora))

1118


In [5]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# Replace with your desired model name
model_name = "all-MiniLM-L6-v2"
model = SentenceTransformerEmbeddings(model_name=model_name)

# Connect to Chroma database (defaults to localhost)
vector_store = Chroma()


/media/warlock/Work/financial models/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/media/warlock/Work/financial models/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/media/warlock/Work/financial models/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new down

In [6]:
db = Chroma.from_documents(corpora,model)

In [8]:
corpora[0]

Document(page_content='The Institute of Chartered Accountants of India\n(Set up by an Act of Parliament)\nNew DelhiISBN  : 978-81-8441-123-2\nwww .icai.orgJuly/2013/5,000(Updated) THE \nCHARTERED ACCOUNTANTS \nACT, 1949\nTHE ACT, 1949 CHARTERED ACCOUNTANTS \n(Updated as on July 1, 2013)\n', metadata={'source': './Acts/pdfs/CA_AmendmentAct.pdf', 'page': 0})

In [7]:
query = "what are requirements for forming a corporation ? "
docs = db.similarity_search(query)

In [9]:
len(docs)

4

In [ ]:
from langchain.prompts import ChatPromptTemplate

temp = "Based on provided context answer following user question : {context} , question : {question}"

prompt_template = ChatPromptTemplate.from_template(temp)

In [ ]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass()

from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest")

In [ ]:
chain = prompt_template | llm

In [ ]:
chain.invoke({"context":docs,"question":"what are requirements for forming a corporation ? "})